This notebook will be a tutorial in how to use Google Search Console API with Python and Pandas.
What we will need:
- Python 3 (will have to download Python 3 or Anaconda)
- pandas
- oauth2Client
- googleapiclient
- apiclient
- httplib2

If you don't have any of these you will need to run pip install (package name) <br>
<br>
Now we import all of our dependencies. This can probably be trimmed, but I did it quickly.

In [2]:
import pandas as pd
import httplib2

from googleapiclient import sample_tools
from googleapiclient import discovery
from googleapiclient.http import build_http
from apiclient import errors
from apiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools

Now we authenticate and get our service

In [3]:
client_secrets = r"Your Secret Filepath Here"
scope = 'https://www.googleapis.com/auth/webmasters.readonly'
# Set up a Flow object to be used if we need to authenticate.
flow = client.flow_from_clientsecrets(
    client_secrets, scope=scope, message=tools.message_if_missing(client_secrets)
    )

# Prepare credentials, and authorize HTTP object with them.
# If the credentials don't exist or are invalid run through the native client
# flow. The Storage object will ensure that if successful the good
# credentials will get written back to a file.
storage = file.Storage('SEO_Project' + ".dat")
credentials = storage.get()
if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage)#, flags)
http = credentials.authorize(http=build_http())
service = discovery.build('searchconsole', 'v1', http=http)

The above code is from google quickstart guide for search console. If you already have your OAuth flow set up, you can use that instead. All we need is the service.

In [4]:
def execute_request(service, property_uri, request):
  """Executes a searchAnalytics.query request.
  Args:
    service: The searchconsole service to use when executing the query.
    property_uri: The site or app URI to request data for.
    request: The request to be executed.
  Returns:
    An array of response rows.
  """
  return service.searchanalytics().query(
      siteUrl=property_uri, body=request).execute()

The above is also from Googles python sample. Now we need to get the total count for the date range of the query we will run. We will start with a small, sample query to verify correct operation. I believe that we get an array output, so we are probably looking for total_rows[1]. However, we will have to see what the print turns up.

In [6]:
property_uri = "https://www.example.com/"

#Date is YYYY-MM-DD format.
start_date = "2020-01-01"
end_date = "2020-01-01"

request = {
     'startDate': start_date,
     'endDate': end_date
    }
total_rows = execute_request(service, property_uri, request)
print(response)

HttpError: <HttpError 403 when requesting https://searchconsole.googleapis.com/webmasters/v3/sites/https%3A%2F%2Fwww.example.com%2F/searchAnalytics/query?alt=json returned "User does not have sufficient permission for site 'https://www.example.com/'. See also: https://support.google.com/webmasters/answer/2451999.". Details: "User does not have sufficient permission for site 'https://www.example.com/'. See also: https://support.google.com/webmasters/answer/2451999.">

Now that we have the total number of rows for our date range, we can execute the request for the data we are actually looking for. I am going to try and pull this into a pandas DataFrame for storage.

In [ ]:
current_row = 0
total_df = pd.DataFrame()
while current_row<tot_rows
    request = {
        'startDate': start_date,
        'endDate': end_date,
        'dimensions': ['query', 'page'],
        'rowLimit': 25000,
        'rowStart':current_row
        }
    current_row = current_row + 25000
    partial_df = pd.DataFrame(execute_request(service, property_uri, request))
    total_df = pd.concat([partial_df,total_df],axis=0, copy=False)

print(total_df.shape)
total_df.head(5)
total_df.to_csv('test_data.csv', index=False)

That should be it! Now we need to implement this for a full year. Ideally, that will mean just changing the date range above. If you run into API call size limitations, that might change. See below for an example of how you would (probably) be able to run a request for each month

In [ ]:
total_rows=[]
for month in range(12):
    days_per_month = [31,28,31,30,31,30,31,31,30,31,30,31,31]
    if month < 9:
        start_date = "2020-0" + str(month+1) + "-01
        end_date = "2020-0" + str(month+2) + "-" + str(days_per_month[month+1])
    elif: month == 9
        start_date = "2020-0" + str(month+1) + "-01
        end_date = "2020-" + str(month+2) + "-" + str(days_per_month[month+1])
    else:
        start_date = "2020-" + str(month+1) + "-01
        end_date = "2020-" + str(month+2) + "-" + str(days_per_month[month+1])
    request = {
         'startDate': start_date,
         'endDate': end_date
        }
    total_rows = execute_request(service, property_uri, request)
    current_row = 0
    total_df = pd.DataFrame()
    while current_row<tot_rows:
        request = {
            'startDate': start_date,
            'endDate': end_date,
            'dimensions': ['query', 'page'],
            'rowLimit': 25000,
            'rowStart':current_row
            }
        current_row = current_row + 25000
        partial_df = pd.DataFrame(execute_request(service, flags.property_uri, request))
        total_df = pd.concat([partial_df,total_df],axis=0, copy=False)

    print(total_df.shape)
    total_df.head(5)
    total_df.to_csv('month_' + str(month+1) + '.csv', index=False)